# 前提環境
Fabric のインストール

In [4]:
!pip install fabric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 555.3 kB/s eta 0:00:001m639.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 756.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 1.5 MB/s eta 0:00:001.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 1.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 2.2 MB/s eta 0:00:002.1 MB/s eta 0:00:01


## Fabric ローカル実行

In [32]:
from fabric import task
from invoke.context import Context
import shutil

@task
def CheckInstallation(c, command):
    result = shutil.which(command)
    print('=== fabfile.main.CheckInstallation()')
    print(f'--> shutil.which: {result}')
    
    return not result is None

CheckInstallation(Context(), 'rustup')

=== fabfile.main.CheckInstallation()
--> shutil.which: None


False

## Rust のインストール

In [1]:
!pwd

/home/tomoyan/my_projects/jupyter-binder/fabric_example


In [11]:
%%file ./fabfile/install_rust.py
from fabric import task
from invoke.watchers import Responder

from . import CheckInstallation, tmux_run

@task
@tmux_run
def Rust_CheckInstalled(c):
    result = CheckInstallation(c, 'rustup')
    return result

@task
@tmux_run
def Rust_Install(c):
    if Rust_CheckInstalled(c):
        print('Rust はインストール済みです😉');
        return False
    
    enterResponder = Responder(
        pattern=r'\n>',
        response='\n'
    )
    result = c.run(
        "curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh",
        hide=False, pty=True,
        watchers=[enterResponder])
    #print(result.stdout.strip())

@task
@tmux_run
def Rust_Uninstall(c):
    if not Rust_CheckInstalled(c):
        print('Rust はインストールされていません🤔');
        return False
    
    yesResponder = Responder(
        pattern=r'Continue\? \(y/N\)',
        response='y\n'
    )
    result = c.run(
        'rustup self uninstall',
        pty=True,
        watchers=[yesResponder]
    )


Overwriting ./fabfile/install_rust.py


In [38]:
!fab --list

args: (<function Rust_CheckInstalled at 0x7f795dbfcd60>,), kwargs: {}
args: (<function Rust_Install at 0x7f795dbfcd60>,), kwargs: {}
args: (<function Rust_Uninstall at 0x7f795dbfcd60>,), kwargs: {}
args: (<function OLS_install at 0x7f795da3b4c0>,), kwargs: {}
args: (<function OLS_uninstall at 0x7f795da3b4c0>,), kwargs: {}
Available tasks:

  CheckInstallation
  ConnectTmux
  OLS-install
  OLS-uninstall
  Rust-CheckInstalled
  Rust-Install
  Rust-Uninstall



In [9]:
!fab Rust-CheckInstalled -d

args: (<function Rust_CheckInstalled at 0x7f6bf7bfcd60>,), kwargs: {}
args: (<function Rust_Install at 0x7f6bf7a7b380>,), kwargs: {}
args: (<function Rust_Uninstall at 0x7f6bf7a7b4c0>,), kwargs: {}
args: (<function OLS_install at 0x7f6bf7a3b4c0>,), kwargs: {}
args: (<function OLS_uninstall at 0x7f6bf7a7b740>,), kwargs: {}
Traceback (most recent call last):
  File "/home/tomoyan/pyJupyterLab/bin/fab", line 8, in <module>
    sys.exit(program.run())
             ^^^^^^^^^^^^^
  File "/home/tomoyan/pyJupyterLab/lib64/python3.11/site-packages/invoke/program.py", line 380, in run
    self.execute()
  File "/home/tomoyan/pyJupyterLab/lib64/python3.11/site-packages/invoke/program.py", line 565, in execute
    executor.execute(*self.tasks)
  File "/home/tomoyan/pyJupyterLab/lib64/python3.11/site-packages/invoke/executor.py", line 127, in execute
    result = call.task(*args, **call.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tomoyan/pyJupyterLab/lib64/python3.11/site-pac

In [36]:
!fab Rust-Install

which: /home/tomoyan/.cargo/bin/rustup
Rust はインストール済みです😉


## Rust のアンインストール

In [38]:
!fab Rust-Uninstall

which: None
Rust はインストールされていません🤔


# 参考文献
https://docs.pyinvoke.org/en/stable/concepts/watchers.html<br>
https://blog.masu-mi.me/post/2021/02/14/catch-up-fabric2/<br>